# PROJETO: MODELAGEM DE FATURAS DO CARTÃO DE CRÉDITO
## Objetivo: Criação de um relatório contendo informações sobre o limite e faturas do cartão de crédito dos clientes

_Desenvolvido por: Raynnie Pereira da Costa_

### Contextualização

A Eras Bank, é uma empresa bancária, em que em seu portfólio de produtos, oferece uma linha de cartão de crédito. 

Atualmente a empresa possui 20 clientes.

Para cada um de seus clientes foi liberado um limite de crédito, definido no momento da solicitação do cartão. 
Sendo que, o limite de cada cliente está cadastrado em uma base de dados.

Assim que eles receberam o cartão, os clientes começaram a utilizá-lo para fazer compras (algumas delas parceladas), e todas estas compras foram registradas em uma outra base de dados.

No entanto, apesar de haver estas duas bases de dados, elas não conversam entre si, o que implica em um grande problema: não é possível saber que clientes gastaram dentro do limite, e quais eventualmente já ultrapassaram o limite!

O desafio do projeto é com o uso de Python, ler os dados disponíveis, realizar o processamento, e no final criar um relatório que mostre qual é o limite de cada cliente, e em quanto está a fatura em cada um dos meses de 2024. 

### Metadados

Há disponível duas bases de dados que estão armazenadas em guias no arquivo Excel "gastos_limites":

* "gastos": contém dados das compras dos clientes.
    * Cada linha se refere a uma única operação.
    * A base é caracterizada pelas colunas: 
        * nm_cliente: nome do cliente que realizou a operação; 
        * valor_operacao: valor da operação; 
        * num_parcelas: número de vezes em que a compra foi parcelada; 
        * dt_operacao: data em que a compra foi realizada; 
        * mes_atual: o mês atual.
    * A base contém apenas apenas operações referentes a janeiro de 2024.

* "limites": contém dados do cadastro de limites de cada um dos clientes.
    * Cada linha se refere a um único cliente.
    * A base é caracterizada pelas colunas:
        * cliente: nome do cliente; 
        * limite: valor do limite de crédito do cartão; 
        * dt_vencimento: data de vencimento da fatura.

### Regra de Negócio

* Data de Fechamento

    Como podemos observar, na base de "limites", apenas temos a data de vencimento da fatura, ou seja, é o último dia para quitar a fatura do cartão de crédito dentro do prazo. 

    A data de fechamento da fatura, por sua vez, é o último dia considerado na hora de calcular o valor a ser pago pelo usuário.
    Comumente, ela ocorre de 7 a 10 dias antes do vencimento da fatura. 

    No caso da Eras Bank, iremos considerar um período de 10 dias. 
    
    Dessa forma, uma fatura que vence no dia 20, terá o fechamento no dia 10, assim, a partir deste dia, tudo que se comprar só será cobrado na próxima fatura.

* Valor da Parcela

    O valor da parcela é calculado da seguinte forma: 
    
    *valor da operação / nº parcelas*

### Leitura dos Dados

##### Importando bibliotecas necessárias

In [724]:
import pandas as pd
import numpy as np
import calendar

##### Lendo as bases de dados

In [725]:
operacao_clientes = pd.read_excel("gastos_limites.xlsx", "gastos")
cadastro_limites = pd.read_excel("gastos_limites.xlsx", "limites")

In [726]:
print("A base de cadastro contém os dados de:", cadastro_limites["cliente"].nunique(), "clientes")
print("A base de operações contém os dados de:", operacao_clientes["nm_cliente"].nunique(), "clientes")

A base de cadastro contém os dados de: 20 clientes
A base de operações contém os dados de: 20 clientes


### Processamento das Bases de Dados

Vamos primeiro ver como estão estruturadas as duas tabelas, para entendermos quais processamentos devem ser aplicados.

In [727]:
operacao_clientes.head() #para visualizar as 5 primeiras linhas da tabela

,nm_cliente,valor_operacao,num_parcelas,dt_operacao,mes_atual
0,Paulo Gomes,"R$ 855,00",5,2024-01-01,2024-01-01
1,Camila Santos,"R$ 504,18",2,2024-01-01,2024-01-01
2,Márcia Silva,"R$ 970,00",5,2024-01-01,2024-01-01
3,Maria Alves,"R$ 2010,31",7,2024-01-01,2024-01-01
4,Lucas Oliveira,"R$ 465,00",1,2024-01-01,2024-01-01


In [728]:
operacao_clientes.dtypes #para visualizar os tipos de dados de cada coluna

nm_cliente                object
valor_operacao            object
num_parcelas               int64
dt_operacao       datetime64[ns]
mes_atual         datetime64[ns]
dtype: object

In [729]:
cadastro_limites.head() #para visualizar as 5 primeiras linhas da tabela

,cliente,limite,dt_vencimento
0,"SILVA, JOÃO","R$ 2575,00",2024-01-25
1,JOSÉ SILVA,"R$ 4500,00",2024-01-05
2,Antônio Oliveira,"R$ 2325,00",2024-01-25
3,"Santos, Francisco","R$ 375,00",2024-01-25
4,"Rodrigues, Carlos","R$ 2675,00",2024-01-05


In [730]:
cadastro_limites.dtypes #para visualizar os tipos de dados de cada coluna

cliente                  object
limite                   object
dt_vencimento    datetime64[ns]
dtype: object

Observa-se alguns tratamentos que precisam ser feitos:
* Normalização do nome dos clientes na base de cadastro
* Ajuste dos tipos de dados monetários em ambas as tabelas, pois estão como texto
* Inserção de chaves primárias para criação de relacionamento entre as tabelas

1. Normalização

O primeiro tratamento que deve ser realizado será em colocar o nome dos clientes em um formato padrão.

Para termos o relacionamento entre as bases, vamos observar como está o formato do nome dos clientes da base de operações.

In [731]:
operacao_clientes["nm_cliente"].unique()

array(['Paulo Gomes', 'Camila Santos', 'Márcia Silva', 'Maria Alves',
       'Lucas Oliveira', 'Sandra Souza', 'Francisco Santos',
       'Antônio Oliveira', 'José Silva', 'Ana Costa', 'Pedro Ribeira',
       'João Silva', 'Patrícia Pereira', 'Marcos Souza',
       'Fernanda Oliveira', 'Carlos Rodrigues', 'Luiz Alves',
       'Aline Rodrigues', 'Juliana Costa', 'Adriana Pereira'],
      dtype=object)

Portanto, precisamos que o nome dos clientes da base de cadastro esteja nesse mesmo formato como "Nome Sobrenome"

In [732]:
cadastro_limites["cliente"].unique()

array(['SILVA, JOÃO', ' JOSÉ SILVA    ', ' Antônio Oliveira    ',
       'Santos, Francisco', 'Rodrigues, Carlos', ' PEDRO RIBEIRA    ',
       ' Paulo Gomes    ', ' LUCAS OLIVEIRA    ', ' LUIZ ALVES    ',
       ' Marcos Souza    ', ' MARIA ALVES    ', ' ANA COSTA    ',
       ' Adriana Pereira    ', 'Costa, Juliana', 'Silva, Márcia',
       ' Fernanda Oliveira    ', ' Patrícia Pereira    ',
       'Rodrigues, Aline', ' SANDRA SOUZA    ', ' Camila Santos    '],
      dtype=object)

In [733]:
cadastro_limites["cliente_ajuste"] = cadastro_limites["cliente"].str.upper() #todas as palavras em caixa alta
cadastro_limites["cliente_ajuste"] = [i.lstrip() for i in cadastro_limites["cliente_ajuste"]] #remove o último espaço a esquerda 
cadastro_limites["cliente_ajuste"] = [i.rstrip() for i in cadastro_limites["cliente_ajuste"]] #remove o último espaço a direita 
cadastro_limites.head()

,cliente,limite,dt_vencimento,cliente_ajuste
0,"SILVA, JOÃO","R$ 2575,00",2024-01-25,"SILVA, JOÃO"
1,JOSÉ SILVA,"R$ 4500,00",2024-01-05,JOSÉ SILVA
2,Antônio Oliveira,"R$ 2325,00",2024-01-25,ANTÔNIO OLIVEIRA
3,"Santos, Francisco","R$ 375,00",2024-01-25,"SANTOS, FRANCISCO"
4,"Rodrigues, Carlos","R$ 2675,00",2024-01-05,"RODRIGUES, CARLOS"


In [734]:
#Lógica: se achar vírgula a palavra anterior é sobrenome, se não achar, a palavra depois do primeiro espaço é sobrenome
cadastro_limites[['Sobrenome','Nome']] = cadastro_limites['cliente_ajuste'].str.split(',',expand=True)
cadastro_limites[['Nome1','Sobrenome1']] = cadastro_limites['Sobrenome'].str.split(' ',expand=True)
cadastro_limites['nome'] = np.where(cadastro_limites['Nome'].isna() , cadastro_limites['Nome1'], cadastro_limites['Nome'])
cadastro_limites['sobrenome'] = np.where(cadastro_limites['Sobrenome1'].isna() , cadastro_limites['Sobrenome'], cadastro_limites['Sobrenome1'])

cadastro_limites['nm_cliente'] = cadastro_limites['nome'].str.title() + " " + cadastro_limites['sobrenome'].str.title()
cadastro_limites["nm_cliente"] = [i.lstrip() for i in cadastro_limites["nm_cliente"]] #remove o último espaço a esquerda 
cadastro_limites["nm_cliente"] = [i.rstrip() for i in cadastro_limites["nm_cliente"]] #remove o último espaço a direita 

cadastro_limites.head()

,cliente,limite,dt_vencimento,cliente_ajuste,Sobrenome,Nome,Nome1,Sobrenome1,nome,sobrenome,nm_cliente
0,"SILVA, JOÃO","R$ 2575,00",2024-01-25,"SILVA, JOÃO",SILVA,JOÃO,SILVA,None,JOÃO,SILVA,João Silva
1,JOSÉ SILVA,"R$ 4500,00",2024-01-05,JOSÉ SILVA,JOSÉ SILVA,None,JOSÉ,SILVA,JOSÉ,SILVA,José Silva
2,Antônio Oliveira,"R$ 2325,00",2024-01-25,ANTÔNIO OLIVEIRA,ANTÔNIO OLIVEIRA,None,ANTÔNIO,OLIVEIRA,ANTÔNIO,OLIVEIRA,Antônio Oliveira
3,"Santos, Francisco","R$ 375,00",2024-01-25,"SANTOS, FRANCISCO",SANTOS,FRANCISCO,SANTOS,None,FRANCISCO,SANTOS,Francisco Santos
4,"Rodrigues, Carlos","R$ 2675,00",2024-01-05,"RODRIGUES, CARLOS",RODRIGUES,CARLOS,RODRIGUES,None,CARLOS,RODRIGUES,Carlos Rodrigues


In [735]:
#Excluindo colunas de processamento
cadastro_limites = cadastro_limites.drop(columns=
['cliente',
 'cliente_ajuste',
 'Sobrenome',
 'Nome',
 'Nome1',
 'Sobrenome1',
 'nome',
 'sobrenome'])

cadastro_limites = cadastro_limites[['nm_cliente', 'limite', 'dt_vencimento']]

cadastro_limites.head()

,nm_cliente,limite,dt_vencimento
0,João Silva,"R$ 2575,00",2024-01-25
1,José Silva,"R$ 4500,00",2024-01-05
2,Antônio Oliveira,"R$ 2325,00",2024-01-25
3,Francisco Santos,"R$ 375,00",2024-01-25
4,Carlos Rodrigues,"R$ 2675,00",2024-01-05


2. Ajuste do Tipo de Dados Monetários

In [736]:
#conversao da coluna de valor_operacao e limite em float
operacao_clientes['valor_operacao'] = operacao_clientes['valor_operacao'].str.replace('R$ ', '').str.replace(',', '.')
operacao_clientes['valor_operacao'] = operacao_clientes['valor_operacao'].astype('float')

cadastro_limites['limite'] = cadastro_limites['limite'].str.replace('R$ ', '').str.replace(',', '.')
cadastro_limites['limite'] = cadastro_limites['limite'].astype('float')

3. Relacionamento entre Tabelas

Tendo em vista que agora ambas as tabelas tem os mesmos formatos dos nomes dos clientes, podemos utilizar essa coluna para relacionar a tabela de cadastro de limites com a tabela de operações para termos uma visão geral.

Contudo, pensando na melhor prática de modelagem de banco de dados, iremos acrescentar chaves primárias no formato de valor inteiro para facilitar a modelagem da solução. 

Dessa forma, iremos incluir as seguintes colunas nas tabelas:
* *id_cliente* na tabela de "cadastro_limites"
* *id_transacao* na tabela de "operacao_clientes"

In [737]:
def criando_id(df, coluna):
    #criando id incremetal que inicia com valor 1
    df[coluna] = df.index + 1                  
    #excluindo coluna recém-criada e armazenando na variável "primeira_coluna"
    primeira_coluna = df.pop(coluna)           
    #inserindo novamente a coluna, porém como primeira coluna da tabela
    df.insert(0, coluna, primeira_coluna)      
    return df

cadastro_limites = criando_id(cadastro_limites, 'id_cliente')
operacao_clientes = criando_id(operacao_clientes, 'id_transacao')

### Aplicação de Regra de Negócio

Vamos juntar ambas as tabelas para aplicação das regras de negócios.

In [738]:
tabela_consumo = pd.merge(operacao_clientes,cadastro_limites,how='inner', on=['nm_cliente','nm_cliente'])
tabela_consumo.head()

,id_transacao,nm_cliente,valor_operacao,num_parcelas,dt_operacao,mes_atual,id_cliente,limite,dt_vencimento
0,1,Paulo Gomes,855.00,5,2024-01-01,2024-01-01,7,4725.0,2024-01-25
1,6,Paulo Gomes,465.50,4,2024-01-02,2024-01-01,7,4725.0,2024-01-25
2,31,Paulo Gomes,343.00,4,2024-01-06,2024-01-01,7,4725.0,2024-01-25
3,33,Paulo Gomes,559.36,2,2024-01-06,2024-01-01,7,4725.0,2024-01-25
4,42,Paulo Gomes,652.44,5,2024-01-08,2024-01-01,7,4725.0,2024-01-25


In [739]:
'''Calculando a data de fechamento, dado a regra de negócio de ser 
10 dias antes da data de vencimento da fatura'''

#criando intervalo de 10 dias
intervalo = pd.Timedelta(days=10)

#retirando os 10 dias da data de vencimento
tabela_consumo['dt_fechamento'] = tabela_consumo['dt_vencimento'] - intervalo
tabela_consumo.head()

,id_transacao,nm_cliente,valor_operacao,num_parcelas,dt_operacao,mes_atual,id_cliente,limite,dt_vencimento,dt_fechamento
0,1,Paulo Gomes,855.00,5,2024-01-01,2024-01-01,7,4725.0,2024-01-25,2024-01-15
1,6,Paulo Gomes,465.50,4,2024-01-02,2024-01-01,7,4725.0,2024-01-25,2024-01-15
2,31,Paulo Gomes,343.00,4,2024-01-06,2024-01-01,7,4725.0,2024-01-25,2024-01-15
3,33,Paulo Gomes,559.36,2,2024-01-06,2024-01-01,7,4725.0,2024-01-25,2024-01-15
4,42,Paulo Gomes,652.44,5,2024-01-08,2024-01-01,7,4725.0,2024-01-25,2024-01-15


In [740]:
'''Calculando valor das parcelas e adicionando em uma lista'''

def valor_parcelas(row):
    return [row['valor_operacao']/row['num_parcelas'] for i in range(row['num_parcelas'])]

#calculando valor das parcelas
tabela_consumo['valor_parcela'] = tabela_consumo.apply(valor_parcelas, axis=1)
tabela_consumo.head()

,id_transacao,nm_cliente,valor_operacao,num_parcelas,dt_operacao,mes_atual,id_cliente,limite,dt_vencimento,dt_fechamento,valor_parcela
0,1,Paulo Gomes,855.00,5,2024-01-01,2024-01-01,7,4725.0,2024-01-25,2024-01-15,"[171.0, 171.0, 171.0, 171.0, 171.0]"
1,6,Paulo Gomes,465.50,4,2024-01-02,2024-01-01,7,4725.0,2024-01-25,2024-01-15,"[116.375, 116.375, 116.375, 116.375]"
2,31,Paulo Gomes,343.00,4,2024-01-06,2024-01-01,7,4725.0,2024-01-25,2024-01-15,"[85.75, 85.75, 85.75, 85.75]"
3,33,Paulo Gomes,559.36,2,2024-01-06,2024-01-01,7,4725.0,2024-01-25,2024-01-15,"[279.68, 279.68]"
4,42,Paulo Gomes,652.44,5,2024-01-08,2024-01-01,7,4725.0,2024-01-25,2024-01-15,"[130.488, 130.488, 130.488, 130.488, 130.488]"


### Modelagem do Relatório 

Dado que adicionamos o valor das parcelas em uma lista, podemos reorganizar o dataframa explodindo a lista em cada parcela.

In [741]:
tabela_fatura = tabela_consumo.explode('valor_parcela')
tabela_fatura = tabela_fatura.drop(['nm_cliente', 'valor_operacao', 'num_parcelas', 'dt_vencimento', 'limite'], axis=1)
tabela_fatura.head()

,id_transacao,dt_operacao,mes_atual,id_cliente,dt_fechamento,valor_parcela
0,1,2024-01-01,2024-01-01,7,2024-01-15,171.0
0,1,2024-01-01,2024-01-01,7,2024-01-15,171.0
0,1,2024-01-01,2024-01-01,7,2024-01-15,171.0
0,1,2024-01-01,2024-01-01,7,2024-01-15,171.0
0,1,2024-01-01,2024-01-01,7,2024-01-15,171.0


In [742]:
#adicionando numero das parcelas
tabela_fatura['num_parcela'] = tabela_fatura.groupby('id_transacao')['dt_operacao'].rank('first').astype('int')
tabela_fatura.head(10)

,id_transacao,dt_operacao,mes_atual,id_cliente,dt_fechamento,valor_parcela,num_parcela
0,1,2024-01-01,2024-01-01,7,2024-01-15,171.0,1
0,1,2024-01-01,2024-01-01,7,2024-01-15,171.0,2
0,1,2024-01-01,2024-01-01,7,2024-01-15,171.0,3
0,1,2024-01-01,2024-01-01,7,2024-01-15,171.0,4
0,1,2024-01-01,2024-01-01,7,2024-01-15,171.0,5
1,6,2024-01-02,2024-01-01,7,2024-01-15,116.375,1
1,6,2024-01-02,2024-01-01,7,2024-01-15,116.375,2
1,6,2024-01-02,2024-01-01,7,2024-01-15,116.375,3
1,6,2024-01-02,2024-01-01,7,2024-01-15,116.375,4
2,31,2024-01-06,2024-01-01,7,2024-01-15,85.75,1


In [743]:
#Ajustando datas para acompanhar os meses das parcelas

def add_meses(coluna_data, num_parcela):
    #Extrair o ano e o dia da data original
    ano = coluna_data.year
    dia = coluna_data.day
    
    #Calcular o novo mês
    novo_mes = coluna_data.month + (num_parcela - 1)
    
    #Ajustar o mês para ficar dentro do intervalo [1, 12]
    novo_mes = (novo_mes - 1) % 12 + 1

    #Calcular o ano resultante
    novo_ano = ano + (novo_mes - 1) // 12
    
    #Obter o último dia válido do novo mês
    ultimo_dia_mes = calendar.monthrange(novo_ano, novo_mes)[1]
    
    #Ajustar o dia para ser válido dentro do mês
    novo_dia = min(dia, ultimo_dia_mes)

    #Criar a nova data
    nova_data = np.datetime64('{:04d}-{:02d}-{:02d}'.format(novo_ano, novo_mes, novo_dia))
    
    return nova_data

tabela_fatura['nova_operacao'] = tabela_fatura.apply(lambda row: add_meses(row['dt_operacao'], row['num_parcela']), axis=1)
tabela_fatura['novo_fechamento'] = tabela_fatura.apply(lambda row: add_meses(row['dt_fechamento'], row['num_parcela']), axis=1)
tabela_fatura['novo_mes_atual'] = tabela_fatura.apply(lambda row: add_meses(row['mes_atual'], row['num_parcela']), axis=1)

tabela_fatura = tabela_fatura.drop(columns=['dt_operacao', 'mes_atual', 'dt_fechamento'])

tabela_fatura.head(10)


,id_transacao,id_cliente,valor_parcela,num_parcela,nova_operacao,novo_fechamento,novo_mes_atual
0,1,7,171.0,1,2024-01-01,2024-01-15,2024-01-01
0,1,7,171.0,2,2024-02-01,2024-02-15,2024-02-01
0,1,7,171.0,3,2024-03-01,2024-03-15,2024-03-01
0,1,7,171.0,4,2024-04-01,2024-04-15,2024-04-01
0,1,7,171.0,5,2024-05-01,2024-05-15,2024-05-01
1,6,7,116.375,1,2024-01-02,2024-01-15,2024-01-01
1,6,7,116.375,2,2024-02-02,2024-02-15,2024-02-01
1,6,7,116.375,3,2024-03-02,2024-03-15,2024-03-01
1,6,7,116.375,4,2024-04-02,2024-04-15,2024-04-01
2,31,7,85.75,1,2024-01-06,2024-01-15,2024-01-01


In [744]:
#Lógica: se data da operacao menor ou igual a data do fechamento: mes_atual da fatura, senão inserir na próxima fatura
def calculo_fatura(row):
    if row['nova_operacao'] <= row['novo_fechamento']:
        mes_fatura = row['novo_mes_atual']
    else:
        mes_fatura = row['novo_mes_atual'] + np.timedelta64(1, 'M')
    return mes_fatura

tabela_fatura['mes_fatura'] = tabela_fatura.apply(calculo_fatura, axis=1)
tabela_fatura.head(10)

,id_transacao,id_cliente,valor_parcela,num_parcela,nova_operacao,novo_fechamento,novo_mes_atual,mes_fatura
0,1,7,171.0,1,2024-01-01,2024-01-15,2024-01-01,2024-01-01
0,1,7,171.0,2,2024-02-01,2024-02-15,2024-02-01,2024-02-01
0,1,7,171.0,3,2024-03-01,2024-03-15,2024-03-01,2024-03-01
0,1,7,171.0,4,2024-04-01,2024-04-15,2024-04-01,2024-04-01
0,1,7,171.0,5,2024-05-01,2024-05-15,2024-05-01,2024-05-01
1,6,7,116.375,1,2024-01-02,2024-01-15,2024-01-01,2024-01-01
1,6,7,116.375,2,2024-02-02,2024-02-15,2024-02-01,2024-02-01
1,6,7,116.375,3,2024-03-02,2024-03-15,2024-03-01,2024-03-01
1,6,7,116.375,4,2024-04-02,2024-04-15,2024-04-01,2024-04-01
2,31,7,85.75,1,2024-01-06,2024-01-15,2024-01-01,2024-01-01


In [745]:
#Ajustar o mes da fatura para constar somente mes_ano
tabela_fatura['mes_fatura'] = tabela_fatura['mes_fatura'].dt.to_period('M')
tabela_fatura.head(2)

,id_transacao,id_cliente,valor_parcela,num_parcela,nova_operacao,novo_fechamento,novo_mes_atual,mes_fatura
0,1,7,171.0,1,2024-01-01,2024-01-15,2024-01-01,2024-01
0,1,7,171.0,2,2024-02-01,2024-02-15,2024-02-01,2024-02


In [746]:
#Agrupando valor das parcelas para saber o valor da fatura total
tabela_fatura_mes = tabela_fatura.groupby(['id_cliente', 'mes_fatura'])['valor_parcela'].sum()
tabela_fatura_mes = tabela_fatura_mes.reset_index()

#Relacionando com tabela de clientes para trazer nome do cliente e o seu limite mensal
relatorio = pd.merge(tabela_fatura_mes,cadastro_limites,how='left', on=['id_cliente','id_cliente'])
relatorio = relatorio.drop(columns=['id_cliente', 'dt_vencimento'])

#Ajustando para o formato de tabela dinâmica para termos a visão dos meses por colunas
relatorio = relatorio.pivot_table(columns='mes_fatura', index=['nm_cliente', 'limite'], values='valor_parcela')

#Ajustando valores para preencher nulos com zero e arredondar valores
relatorio = relatorio.fillna(0)
relatorio = relatorio.round(2)
relatorio = relatorio.reset_index()

relatorio

mes_fatura,nm_cliente,limite,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01
0,Adriana Pereira,1875.0,0.00,4015.78,1895.90,1163.48,147.35,108.46,108.46,108.46,108.46,108.46,108.46,0.00,0.00
1,Aline Rodrigues,2600.0,0.00,1197.12,113.00,113.00,113.00,113.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,Ana Costa,5375.0,615.96,1004.92,1004.92,1004.92,556.39,225.04,123.73,123.73,123.73,123.73,0.00,0.00,0.00
3,Antônio Oliveira,2325.0,1120.75,2045.17,905.93,632.67,228.62,95.70,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,Camila Santos,3275.0,295.48,2749.15,678.06,345.06,43.39,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,Carlos Rodrigues,2675.0,0.00,1147.23,1147.23,965.23,965.23,879.46,131.73,131.73,0.00,0.00,0.00,0.00,0.00
6,Fernanda Oliveira,2925.0,0.00,950.18,950.18,666.13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,Francisco Santos,375.0,1416.80,1824.01,461.50,461.50,366.60,284.40,249.18,88.90,88.90,88.90,0.00,0.00,0.00
8,José Silva,4500.0,0.00,2268.55,498.71,405.11,156.77,76.28,76.28,76.28,76.28,76.28,76.28,76.28,0.00
9,João Silva,2575.0,188.81,1644.08,900.08,843.08,843.08,140.12,140.12,140.12,0.00,0.00,0.00,0.00,0.00


#### Exportando Relatório

In [747]:
relatorio.to_excel("Relatorio_Faturas.xlsx", index=False)

##### Observação

Existem passos na resolução do relatório que poderiam ser melhor desenvolvidos, além de outros elementos que poderiam ser adicionados, como:

* Análises adicionais e gráficos para facilitar visualização do consumo dos clientes
* Formato de visualização do relatório (valor da fatura com formato R$)